In [12]:
import nltk
nltk.download('wordnet')
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
sno = nltk.stem.SnowballStemmer('english')

In [15]:
lemmatizer = WordNetLemmatizer()

In [16]:
def remove_punc(string):

	punctuations = '''-⎡⎤⎣⎦⎢πε−’–×÷!→ˆ•−–′‘“”\n₹€£()-=+–’[]{};:'"\,<>./?@#$%^&*_~`?.∫'''

	for x in string:
		if x in punctuations:
			string = string.replace(x, " ")

	return re.sub(' +', ' ',string.lower().strip())

In [22]:
with open("corpus.txt",'r') as f1:
    docs = f1.readlines()
with open("query.txt",'r') as f2:
    queries = f2.readlines()

doc_0=remove_punc(docs[0])
print(doc_0)
word_list = nltk.word_tokenize(doc_0)
doc_0 = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
print(doc_0)

in physics a force is an influence that can change the motion of an object a force can cause an object with mass to change its velocity e g moving from a state of rest i e to accelerate force can also be described intuitively as a push or a pull a force has both magnitude and direction making it a vector quantity it is measured in the si unit of newton n force is represented by the symbol f formerly p the original form of newton s second law states that the net force acting upon an object is equal to the rate at which its momentum changes with time if the mass of the object is constant this law implies that the acceleration of an object is directly proportional to the net force acting on the object is in the direction of the net force and is inversely proportional to the mass of the object concepts related to force include thrust which increases the velocity of an object drag which decreases the velocity of an object and torque which produces changes in rotational speed of an object in

In [23]:
corpus = []
query = []
for doc in docs:
    tmp = remove_punc(doc)
    word_list = nltk.word_tokenize(tmp)
    lemmas = [lemmatizer.lemmatize(w) for w in word_list] # lemmatization
    stemmed = " ".join([sno.stem(w) for w in lemmas if w not in stopwords.words('english')]) # stemming of lemmas
    corpus.append(stemmed)

    
for q in queries:
    tmp = remove_punc(q)
    word_list = nltk.word_tokenize(tmp)
    lemmas = [lemmatizer.lemmatize(w) for w in word_list] # lemmatization
    stemmed = " ".join([sno.stem(w) for w in lemmas if w not in stopwords.words('english')])
    query.append(stemmed)

## Unigram model

In [24]:
for q in query:
    q_words = nltk.word_tokenize(q)
    prob = []
    for doc in corpus:
        doc_words = nltk.word_tokenize(doc)
        d = []
        p = 1
        for word in q_words:
            p *= doc_words.count(word) / len(doc_words) + 1e-7
        prob.append(p)
    max_prob_idx = np.argmax(prob)
    print(f'query is: {q}')
    print(f'relevant doc: {corpus[max_prob_idx]}')
    print(prob)

query is: python iter languag
relevant doc: python interpret high level general purpos program languag design philosophi emphas code readabl use signific indent languag construct well object orient approach aim help programm write clear logic code small larg scale project python dynam type garbag collect support multipl program paradigm includ structur particular procedur object orient function program often describ batteri includ languag due comprehens standard librari guido van rossum began work python late 1980s successor abc program languag first releas 1991 python 0 9 0 python 2 0 wa releas 2000 introduc new featur list comprehens cycl detect garbag collect system addit refer count python 3 0 wa releas 2008 wa major revis languag complet backward compat python 2 wa discontinu version 2 7 18 2020 python consist rank one popular program languag
[9.999999999999997e-22, 7.407507407407407e-17, 3.0234427059812774e-10, 8.849657522123892e-17, 9.999999999999997e-22, 9.999999999999997e-22]


# Bigram model

In [25]:
corpus = []
query = []
for doc in docs:
    tmp = remove_punc(doc)
    word_list = nltk.word_tokenize(tmp)
    lemmas = [lemmatizer.lemmatize(w) for w in word_list] # lemmatization
    stemmed = [sno.stem(w) for w in lemmas if w not in stopwords.words('english')]
    corpus.append(stemmed)

    
for q in queries:
    tmp = remove_punc(q)
    word_list = nltk.word_tokenize(tmp)
    
    lemmas = [lemmatizer.lemmatize(w) for w in word_list] # lemmatization
    stemmed = [sno.stem(w) for w in lemmas if w not in stopwords.words('english')]
    query.append(stemmed)

In [26]:
def create_bigrams(words):
    """
    Function that creates a list of bigrams from a list of words in a document.
    Returns a list.
    """
    bigrams = []
    
    for word1, word2 in zip(words, words[1:]):
        bigrams.append(" ".join([word1, word2]))
    return bigrams

In [11]:
for q in query:
    query_bigrams = create_bigrams(q)
    
    prob = []
    for doc in corpus:
        doc_bigrams = create_bigrams(doc)      
        p = 1
        for bigram in query_bigrams:
            p *= doc_bigrams.count(bigram) / len(doc_bigrams) + 1e-7
            
        prob.append(p)
    max_prob_idx = np.argmax(prob)
    
    q = " ".join(q)
    d = " ".join(corpus[max_prob_idx])
    print(f'query is: {q}')
    print(f'relevant doc: {d}')
    print(prob)
    print()

query is: python iter languag
relevant doc: physic forc influenc chang motion object forc caus object mass chang veloc e g move state rest e acceler forc also describ intuit push pull forc ha magnitud direct make vector quantiti measur si unit newton n forc repres symbol f former p origin form newton second law state net forc act upon object equal rate momentum chang time mass object constant law impli acceler object direct proport net forc act object direct net forc invers proport mass object concept relat forc includ thrust increas veloc object drag decreas veloc object torqu produc chang rotat speed object extend bodi part usual appli forc adjac part distribut forc bodi intern mechan stress intern mechan stress caus acceler bodi forc balanc one anoth pressur distribut mani small forc appli area bodi simpl type stress unbalanc caus bodi acceler stress usual caus deform solid materi flow fluid
[9.999999999999998e-15, 9.999999999999998e-15, 9.999999999999998e-15, 9.999999999999998e-15,